<a href="https://colab.research.google.com/github/Avatar2001/AI-Procurement-Assistant/blob/main/AI_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install -qU crewai[tools,agentops]

In [15]:
!pip install -qU tavily-python

In [16]:
!pip install scrapegraph-py

In [17]:
from crewai import Agent,Task,Crew,Process,LLM
from crewai.tools import tool
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
import agentops
import os
from google.colab import userdata
from pydantic import BaseModel,Field
from typing import List
from tavily import TavilyClient
from scrapegraph_py import Client
import json

os.environ["GOOGLE_API_KEY"]=userdata.get('GOOGLE_API_KEY')
os.environ['AGENTOPS_API_KEY']=userdata.get('agentops.api_key')
os.environ['TAVILY_API_KEY']=userdata.get('TAVILY_API_KEY')
os.environ['SCRAPEGRAPH_API_KEY']=userdata.get('SCRAPEGRAPH_API_KEY')

agentops.init(
    api_key=userdata.get('agentops.api_key'),
    skip_auto_end_session=True
)

In [18]:
output_dir = "./ai-agent-output"
os.makedirs(output_dir, exist_ok=True)

basic_llm = LLM(model="gemini/gemini-2.0-flash", temperature=0,api_key=userdata.get('GOOGLE_API_KEY'))
search_client=TavilyClient(api_key=userdata.get('TAVILY_API_KEY'))
scrape_client=Client(api_key=userdata.get('SCRAPEGRAPH_API_KEY'))

In [19]:
about_company = "Gamed Gamed is a company that provides AI solutions to help websites refine their search and recommendation systems."

company_context = StringKnowledgeSource(
    content=about_company
)

In [20]:
no_keywords=10

class SuggestedSearchQueries(BaseModel):
    queries: List[str]=Field(description="A list of suggested search queries to be passed to the search engine.",
                             min_items=1,
                             max_items=no_keywords)


Search_Queries_Recommendation_agent=Agent(
    role="Search Queries Reccomndation Agent",
    goal="\n".join(["To provide a list of suggested search queries to be passed to the search engine.",
    "the queries must be varied and looking for specific items."]),

    backstory="the agent is designed to help in looking for products by providing a list of suggested search queries to be passed to the search engine based on the context povide.",
    llm=basic_llm,
    verbose=True
)

Search_Queries_Recommendation_taskk=Task(
    description="\n".join([
        "company is looking to buy {product_name} at the best prices (value for a price strategy)",
        "the company target any of these websites to buy from:{websites_list}",
        "the company wants to reach all available poducts on the intenet to be compared later in another stage.",
        "the stores must sell the poduct in {country_name}",
        "Generate only {no_keywords} querries.",
        "The search keywords must be in {language} language.",
        "Search keywods could mention specific brands, types o technologies",
        "The search query must an ecommerce webpage for product, and not a blog or listing page."



    ]),
    expected_output="A JSON object containing a list of suggested search queries to be passed to the search engine.",
    output_json=SuggestedSearchQueries,
    output_file=os.path.join(output_dir,"search_queries.json"),
    agent=Search_Queries_Recommendation_agent
)

In [21]:
class SingleSeachResult(BaseModel):
    title:str
    link:str
    content:str
    score:float
    search_query:str

class AllSearchResults(BaseModel):
    results:List[SingleSeachResult]


@tool
def search_engine_tool(query:str):
  """
  Useful for search-based queries. Use this to find current information about any query related pages using a search engine.

   """

  return search_client.search(query)

search_engine_agent=Agent(
    role="Search Engine Agent",
    goal="To search for products based on the suggested search query",
    backstory="the agent is designed to help in searching for products based on the suggested search query.",
    llm=basic_llm,
    verbose=True,
    tools=[search_engine_tool]
)

search_engine_task=Task(
    description="\n".join([
        "the task is to search for products based on the suggested search query.",
        "You have to collect results from multpile search queries.",
        "Ignore any susbicious links or not an ecommerce single website link.",
        "Ignore any search results with confidence score less than ({score_threeshold}).",
        "The search result will be used to compare prices of products from diffferent websites."
    ]),
    expected_output="A JSON object containing the search results.",
    output_json=AllSearchResults,
    output_file=os.path.join(output_dir,"search_results.json"),
    agent=search_engine_agent
)



In [22]:
class ProductSpecs(BaseModel):
    specification_name: str
    specification_value: str


class SingleExtractedProduct(BaseModel):
    page_url: str = Field(..., title="The original URL of the product page")
    product_title: str = Field(..., title="The title of the product")
    product_image_url: str = Field(..., title="The image URL of the product")
    product_url: str = Field(..., title="The URL of the product")
    product_current_price: float = Field(..., title="The current price of the product")
    product_original_price: float = Field(None, title="The original price of the product")
    product_discount_percentage: float = Field(None, title="The discount percentage of the product")
    product_description: str = Field(..., title="The description of the product")

    product_specs: List[ProductSpecs] = Field(
        ...,
        title="The specifications of the product, Focus on the most important specs to compare.",
        min_items=1,
        max_items=5
    )
    agent_recommendation_rank: int = Field(
        ...,
        title="The rank of the product (out of 5, Higher is better) in the recommendation list ordering from the best to the worst"
    )
    agent_recommendation_notes: str = Field(
        ...,
        title="A set of notes why would you recommend or not recommend this product to the company, compared to other products."
    )


class AllExtractedProducts(BaseModel):
    products: List[SingleExtractedProduct]


@tool
def Web_Scraping_tool(page_url:str):
  """
  An AI Tool to help an agent to scrape a web page

  Example:
  Web_Scraping_tool(

    page_url="https://www.amazon.eg/
    required_fields=["title","price","description"]
   )


  """

  details=scrape_client.smartscraper(
      website_url=page_url,
      user_prompt="Extact '''Json\n"+SingleExtractedProduct.schema_json()+"\'''n from the web page"
  )
  return {
      "page_url":page_url,
      "details":details

  }

scraping_agent=Agent(
    role="Web scraping agent",
    goal="To extract details from any website",
    backstory="The agent is designed to help in looking for required values from any website url. These details will be used to decide which best product to buy",
    llm=basic_llm,
    verbose=True,
    tools=[Web_Scraping_tool]

)

scraping_task=Task(
    description="\n".join([
        "The task is to extract products details from any ecommerce store page url.",
        "The task has to collect results from multpile pages urls.",
        "Collect the best {top_recommendations_no} products from the search results.",

    ]),
    expected_output="A JSON object containing the extracted details.",
    output_json=AllExtractedProducts,
    agent=scraping_agent,
    output_file=os.path.join(output_dir,"extracted_details.json"),
)

In [23]:
proccurement_report_author_agent=Agent(
    role="Procurement Report Author Agent",
    goal="To generate a professional HTML page for the procurement report.",
    backstory="The agent is designed to help in generating a professional HTML page for the procurement report after looking into a list of products.",
    llm=basic_llm,
    verbose=True,

)

proccurement_report_author_task=Task(
    description="\n".join([
        "The task is to generate a professional HTML page for the procurement report.",
        "You have to use Bootstrap CSS framework for a better UI.",
        "Use the provided context about the company to make a specialized report.",
        "The report will include the search results and prices of products from different websites.",
        "The report should be structured with the following sections:",
        "1. Executive Summary: A brief overview of the procurement process and key findings.",
        "2. Introduction: An introduction to the purpose and scope of the report.",
        "3. Methodology: A description of the methods used to gather and compare prices.",
        "4. Findings: Detailed comparison of prices from different websites, including tables and charts.",
        "5. Analysis: An analysis of the findings, highlighting any significant trends or observations.",
        "6. Recommendations: Suggestions for procurement based on the analysis.",
        "7. Conclusion: A summary of the report and final thoughts.",
        "8. Appendices: Any additional information, such as raw data or supplementary materials.",

]),
    expected_output="A professional HTML page for the proccurement report.",
    output_file=os.path.join(output_dir,"proccurement_report.html"),
    agent=proccurement_report_author_agent
)


In [24]:
report_verifier_agent = Agent(
    role="Procurement Report Verifier Agent",
    goal="To verify the quality and completeness of the procurement report HTML.",
    backstory="This agent ensures the procurement report is professional, complete, visually clear, and adheres to HTML best practices including Bootstrap usage.",
    llm=basic_llm,
    verbose=True
)
report_verifier_task = Task(
    description="\n".join([
        "The task is to verify the generated HTML page for the procurement report.",
        "Check for the following:",
        "- All 8 required sections are present and clearly labeled.",
        "- Bootstrap CSS framework is used properly for responsive layout and styling.",
        "- The content is coherent, well-structured, and reflects the analysis correctly.",
        "- Tables and charts are readable and styled professionally.",
        "- There are no broken tags or major formatting issues.",
        "- Recommendations and analysis are logically derived from findings.",
        "- The HTML follows best practices and is valid.",
        "Provide a detailed verification summary at the top and insert inline comments where improvements are needed."
    ]),
    expected_output="A verified and annotated HTML file, with comments or a verification report at the top of the file.",
    agent=report_verifier_agent,
    input_file=os.path.join(output_dir, "proccurement_report.html"),
    output_file=os.path.join(output_dir, "verified_proccurement_report.html")
)


In [25]:
company=Crew(
    agents=[
        Search_Queries_Recommendation_agent,
        search_engine_agent,
        scraping_agent,
        proccurement_report_author_agent,
        report_verifier_agent
        ],
    tasks=[
        Search_Queries_Recommendation_taskk,
        search_engine_task,
        scraping_task,
        proccurement_report_author_task,
        report_verifier_task
        ],
    process=Process.sequential,
    knowledge_sources=[company_context]
)
crew_result=company.kickoff(
    inputs={
        "product_name":"coffe machine for the office",
        "websites_list":["www.amazon.eg","www.jumia.com.eg","www.noon.com/egypt-en"],
        "country_name":"Egypt",
        "no_keywords":no_keywords,
        "language":"Arabic",
        "score_threeshold":0.10,
        "top_recommendations_no":10

    }

)

# Agent: Search Queries Reccomndation Agent
## Task: company is looking to buy coffe machine for the office at the best prices (value for a price strategy)
the company target any of these websites to buy from:['www.amazon.eg', 'www.jumia.com.eg', 'www.noon.com/egypt-en']
the company wants to reach all available poducts on the intenet to be compared later in another stage.
the stores must sell the poduct in Egypt
Generate only 10 querries.
The search keywords must be in Arabic language.
Search keywods could mention specific brands, types o technologies
The search query must an ecommerce webpage for product, and not a blog or listing page.


# Agent: Search Queries Reccomndation Agent
## Final Answer: 
```json
{
  "queries": [
    "ماكينة قهوة اسبريسو منزلية مصر",
    "ماكينة قهوة بالكبسولات أمازون مصر",
    "ماكينة قهوة فيليبس اوتوماتيك مصر",
    "ماكينة قهوة ديلونجي للبيع في مصر",
    "اسعار ماكينات القهوة في مصر",
    "أفضل ماكينة قهوة للمكاتب مصر",
    "ماكينة قهوة تركي كهربائية مصر"

<ipython-input-22-0e7b7306cced>:53: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  user_prompt="Extact '''Json\n"+SingleExtractedProduct.schema_json()+"\'''n from the web page"


 

I encountered an error while trying to use the tool. This was the error: [402] Insufficient credits.
 Tool Web_Scraping_tool accepts these inputs: Tool Name: Web_Scraping_tool
Tool Arguments: {'page_url': {'description': None, 'type': 'str'}}
Tool Description: 
  An AI Tool to help an agent to scrape a web page

  Example:
  Web_Scraping_tool(

    page_url="https://www.amazon.eg/
    required_fields=["title","price","description"]
   )


  



# Agent: Web scraping agent
## Thought: I need to extract product details from the given URLs. Since I have multiple URLs to process, I will iterate through them and use the `Web_Scraping_tool` to extract the required information from each page. After extracting the data, I will format it into a JSON object as requested.
## Using tool: Web_Scraping_tool
## Tool Input: 
"{\"page_url\": \"https://www.amazon.eg/\\u0645\\u0627\\u0643\\u064a\\u0646\\u0627\\u062a-\\u0627\\u0644\\u0625\\u0633\\u0628\\u0631\\u0633\\u0648-\\u0648\\u0627\\u0644\\u0643\

🖇 AgentOps: Session Replay for default.session trace: https://app.agentops.ai/sessions?trace_id=f091cd3f24ab78dcfd827f717f2a62f3
